In [13]:
import sys
sys.path.append("../base")

import boilerplate
from pyrtl import *

import z3
z3.set_param(proof=True)

In [18]:
reset_working_block()

pc = Register(name='pc', bitwidth=4)
sp = Register(name='sp', bitwidth=4)
mem = MemBlock(name='mem', bitwidth=4, addrwidth=4)
rom = RomBlock(name='rom', bitwidth=4, addrwidth=4, romdata=[0x1, 0x5, 0x1, 0x8, 0x2, 0xf],
               asynchronous=True, pad_with_zeros=True)

out = Output(name='out', bitwidth=4)

In [19]:
instr = rom[pc]
earg = rom[(pc+1)[:4]]

with conditional_assignment:
    with instr == 0x1:   # PUSH
        mem[sp] |= earg
        sp.next |= sp + 1
        pc.next |= pc + 2
    with instr == 0x2:      # POP
        sp.next |= sp - 1
        pc.next |= pc + 1

out <<= mem[sp]

In [20]:
sim = Simulation()
for i in range(10):
    sim.step({})

In [21]:
sim.tracer.render_trace()

<IPython.core.display.Javascript object>

In [22]:
sim.tracer.print_trace()

--- Values in base 10 ---
out 0 0 0 8 8 8 8 8 8 8
pc  0 2 4 5 5 5 5 5 5 5
sp  0 1 2 1 1 1 1 1 1 1


In [23]:
from circuit import net_to_smt

wires, ops, tr = net_to_smt(working_block(), mems=[mem])

In [24]:
from presentation_forms import vertically
vertically(tr)

"tmp178 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0))"
"tmp157 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0))"
"tmp165 = Concat(Extract(0, 0, 0), Extract(0, 0, 0))"
"tmp158 = Concat(tmp157, 1)"
"tmp166 = Concat(tmp165, 2)"
"tmp169 = Concat(Extract(0, 0, 0), Extract(0, 0, 0))"
tmp156 = rom[pc]
tmp189 = mem[sp]
"tmp167 = Concat(0, pc) + Concat(0, tmp166)"
"tmp152 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0))"
"tmp161 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0))"


In [25]:
# you can find this implementation in `verification_utils` as well
def mk_bv_array(bitwidth, addrwidth, data):
    a = z3.K(z3.BitVecSort(addrwidth), z3.BitVecVal(0, bitwidth))
    for i, d in enumerate(data):
        a = z3.Store(a, i, d)
    return a

In [31]:

from verification_utils import CHCs, mk_bv_array

state_vars = [wires.lookup(v) for v in ['pc', 'sp', 'mem']]
Inv = z3.Function("Inv", *(v.sort() for v in state_vars), z3.BoolSort())

vrom = wires.lookup_mem('rom')
crom = mk_bv_array(4, 4, [0x1, 0x5, 0x1, 0x8, 0x2, 0xf])

def pre_post():
    pc, sp, mem = state_vars
    out = mem[sp]

    pre = z3.And(pc == 0, sp == 0, mem == mk_bv_array(4, 4, []))
    post = z3.Implies(crom[pc] == 0xf, out == 8)

    return pre, post

def create_rules(pre, post):
    from z3 import Implies, And, Or, Not
            
    rom_eq = (vrom == crom)
    
    sigma = state_vars
    sigma_p = [ops.primed(v) for v in state_vars]
    
    start = Implies(pre, Inv(sigma))
    step = Implies(And(Inv(sigma), rom_eq, *tr), Inv(sigma_p))
    end = Implies(And(Inv(sigma), Not(post)), False)
        
    return CHCs([start, step, end])

rules = create_rules(*pre_post())
rules

"pc = 0 ∧ sp = 0 ∧ mem = K(BitVec(4), 0) ⇒ Inv(pc, sp, mem)"
"Inv(pc, sp, mem) ∧ rom = Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15) ∧ tmp178 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp157 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp165 = Concat(Extract(0, 0, 0), Extract(0, 0, 0)) ∧ tmp158 = Concat(tmp157, 1) ∧ tmp166 = Concat(tmp165, 2) ∧ tmp169 = Concat(Extract(0, 0, 0), Extract(0, 0, 0)) ∧ tmp156 = rom[pc] ∧ tmp189 = mem[sp] ∧ tmp167 = Concat(0, pc) + Concat(0, tmp166) ∧ tmp152 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp161 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp172 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp179 = Concat(tmp178, 1) ∧ tmp153 = Concat(tmp152, 1) ∧ tmp159 = If(tmp156 = tmp158, 1, 0) ∧ tmp170 = Concat(tmp169, 2) ∧ tmp182 = ~tmp159 ∧ tmp184 = If(0 = tmp159, 0, 1) ∧ tmp176 = ~tmp159 ∧ tmp162 = Concat(tmp161, 1) ∧ out = tmp189 ∧ tmp171 = If(tmp156 = tmp170, 1, 0) ∧ tmp154 = Concat(0, pc) + Concat(0, tmp153) ∧ tmp180 = Concat(0, pc) + Concat(0, tmp179) ∧ tmp173 = Concat(tmp172, 1) ∧ tmp168 = Extract(3, 0, tmp167) ∧ tmp187 = If(0 = tmp159, pc, tmp168) ∧ tmp163 = Concat(0, sp) + Concat(0, tmp162) ∧ tmp155 = Extract(3, 0, tmp154) ∧ tmp177 = tmp176 & tmp171 ∧ tmp183 = tmp182 & tmp171 ∧ tmp181 = Extract(3, 0, tmp180) ∧ tmp174 = Concat(0, sp) - Concat(0, tmp173) ∧ tmp160 = rom[tmp155] ∧ tmp164 = Extract(3, 0, tmp163) ∧ mem:wport:sp = memwr(sp, tmp160, tmp184 = 1) ∧ tmp185 = If(0 = tmp159, sp, tmp164) ∧ tmp188 = If(0 = tmp183, tmp187, tmp181) ∧ tmp175 = Extract(3, 0, tmp174) ∧ pc:next = tmp188 ∧ tmp186 = If(0 = tmp177, tmp185, tmp175) ∧ sp:next = tmp186 ∧ mem:next = If(wr_en(mem:wport:sp), Store(mem, addr(mem:wport:sp), data(mem:wport:sp)), mem) ⇒ Inv(pc:next, sp:next, mem:next)"
"Inv(pc, sp, mem) ∧ ¬(Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15)[pc] = 15 ⇒ mem[sp] = 8) ⇒ False"


**Solve the constraints.**

_If the result is_ `sat`, _this will display the inferred loop invariant_

In [30]:
%time res = rules.solve()
res

CPU times: user 253 ms, sys: 5.09 ms, total: 258 ms
Wall time: 260 ms


Inv,"(Extract(0, 0, ν0) = 1 ∨ ¬(Extract(2, 2, ν0) = 1) ∨ ν2[15 + ν1] = 8) ∧ (¬(Extract(0, 0, ν0) = 1) ∨ ν2[ν1] = 8)"


_If the result is_ `unsat`, _a counterexample trace can be extracted from the proof of unsatisfiability._

In [34]:
def pre_post():
    pc, sp, mem = state_vars
    out = mem[sp]

    pre = z3.And(pc == 0, sp == 0, mem == mk_bv_array(4, 4, []))
    post = z3.Implies(crom[pc] == 0xf, out == 7)

    return pre, post

chcs = create_rules(*pre_post())
chcs

"pc = 0 ∧ sp = 0 ∧ mem = K(BitVec(4), 0) ⇒ Inv(pc, sp, mem)"
"Inv(pc, sp, mem) ∧ rom = Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15) ∧ tmp178 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp157 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp165 = Concat(Extract(0, 0, 0), Extract(0, 0, 0)) ∧ tmp158 = Concat(tmp157, 1) ∧ tmp166 = Concat(tmp165, 2) ∧ tmp169 = Concat(Extract(0, 0, 0), Extract(0, 0, 0)) ∧ tmp156 = rom[pc] ∧ tmp189 = mem[sp] ∧ tmp167 = Concat(0, pc) + Concat(0, tmp166) ∧ tmp152 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp161 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp172 = Concat(Concat(Extract(0, 0, 0), Extract(0, 0, 0)), Extract(0, 0, 0)) ∧ tmp179 = Concat(tmp178, 1) ∧ tmp153 = Concat(tmp152, 1) ∧ tmp159 = If(tmp156 = tmp158, 1, 0) ∧ tmp170 = Concat(tmp169, 2) ∧ tmp182 = ~tmp159 ∧ tmp184 = If(0 = tmp159, 0, 1) ∧ tmp176 = ~tmp159 ∧ tmp162 = Concat(tmp161, 1) ∧ out = tmp189 ∧ tmp171 = If(tmp156 = tmp170, 1, 0) ∧ tmp154 = Concat(0, pc) + Concat(0, tmp153) ∧ tmp180 = Concat(0, pc) + Concat(0, tmp179) ∧ tmp173 = Concat(tmp172, 1) ∧ tmp168 = Extract(3, 0, tmp167) ∧ tmp187 = If(0 = tmp159, pc, tmp168) ∧ tmp163 = Concat(0, sp) + Concat(0, tmp162) ∧ tmp155 = Extract(3, 0, tmp154) ∧ tmp177 = tmp176 & tmp171 ∧ tmp183 = tmp182 & tmp171 ∧ tmp181 = Extract(3, 0, tmp180) ∧ tmp174 = Concat(0, sp) - Concat(0, tmp173) ∧ tmp160 = rom[tmp155] ∧ tmp164 = Extract(3, 0, tmp163) ∧ mem:wport:sp = memwr(sp, tmp160, tmp184 = 1) ∧ tmp185 = If(0 = tmp159, sp, tmp164) ∧ tmp188 = If(0 = tmp183, tmp187, tmp181) ∧ tmp175 = Extract(3, 0, tmp174) ∧ pc:next = tmp188 ∧ tmp186 = If(0 = tmp177, tmp185, tmp175) ∧ sp:next = tmp186 ∧ mem:next = If(wr_en(mem:wport:sp), Store(mem, addr(mem:wport:sp), data(mem:wport:sp)), mem) ⇒ Inv(pc:next, sp:next, mem:next)"
"Inv(pc, sp, mem) ∧ ¬(Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15)[pc] = 15 ⇒ mem[sp] = 7) ⇒ False"


In [35]:
chcs.solve()

0,False
1,"query!170(5, 1, Store(Store(K(BitVec(4), 0), 1, 8), 0, 5))"
2,"∀A, B, C : Inv(A, B, C) ∧ ¬(C[B] = 7) ∧ Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15)[A] = 15 ⇒ query!170(A, B, C)"
3,"Inv(5, 1, Store(Store(K(BitVec(4), 0), 1, 8), 0, 5))"
4,"∀A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, A1, B1 : Inv(A, B, C) ∧ W = memwr(B, U, G = 1) ∧ J = If(D = 2, 1, 0) ∧ Q = ~(~I | ~J) ∧ H = ~G ∧ R = ~(~H | ~J) ∧ I = ~G ∧ G = If(D = 1, 1, 0) ∧ T = 31 + Concat(0, B) ∧ K = 1 + Concat(0, A) ∧ L = 1 + Concat(0, A) ∧ O = 1 + Concat(0, B) ∧ F = 2 + Concat(0, A) ∧ D = Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15)[A] ∧ P = Extract(3, 0, K) ∧ U = Store(Store(Store(Store(Store(Store(K(BitVec(4), 0), 0, 1), 1, 5), 2, 1), 3, 8), 4, 2), 5, 15)[P] ∧ M = Extract(3, 0, F) ∧ Y = If(R = 0, N, S) ∧ X = If(G = 0, B, V) ∧ V = Extract(3, 0, O) ∧ S = Extract(3, 0, L) ∧ N = If(G = 0, A, M) ∧ E = C[B] ∧ A1 = If(Q = 0, X, Z) ∧ Z = Extract(3, 0, T) ∧ B1 = If(wr_en(W), Store(C, addr(W), data(W)), C) ⇒ Inv(Y, A1, B1)"
5,"Inv(4, 2, Store(Store(K(BitVec(4), 0), 0, 5), 1, 8))"
6,"Inv(2, 1, Store(K(BitVec(4), 0), 0, 5))"
7,"Inv(0, 0, K(BitVec(4), 0))"
8,"query!170(5, 1, Store(Store(K(BitVec(4), 0), 1, 8), 0, 5)) ⇒ False"
